In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta
from dateutil import parser
import time
import random
import pymongo
from dotenv import load_dotenv
import os
import scraper_faz

In [ ]:
load_dotenv()
mongodb_uri = os.getenv("MONGODB_URI")

In [169]:
mongoclient = pymongo.MongoClient(mongodb_uri)
mongodb = mongoclient["media-scraper"]
mongo_execs = mongodb["executions"]
mongo_articles = mongodb["articles"]

In [170]:
execution = {}

In [172]:
articles = scraper_faz.request_articles(execution)

Requesting:  https://www.faz.net/suche/?query=&ct=article&ct=blog&ct=storytelling&author=&from=11.11.2023&to=11.11.2023
Articles count:  19
Found a next page. Requesting:  https://www.faz.net/suche/s2.html?ct=article&ct=blog&ct=storytelling&&from=11.11.2023&to=11.11.2023#listPagination
Requesting:  https://www.faz.net/suche/s2.html?ct=article&ct=blog&ct=storytelling&&from=11.11.2023&to=11.11.2023
Articles count:  19
Found a next page. Requesting:  https://www.faz.net/suche/s3.html?ct=article&ct=blog&ct=storytelling&&from=11.11.2023&to=11.11.2023#listPagination
Requesting:  https://www.faz.net/suche/s3.html?ct=article&ct=blog&ct=storytelling&&from=11.11.2023&to=11.11.2023
Articles count:  19
Found a next page. Requesting:  https://www.faz.net/suche/s4.html?ct=article&ct=blog&ct=storytelling&&from=11.11.2023&to=11.11.2023#listPagination
Requesting:  https://www.faz.net/suche/s4.html?ct=article&ct=blog&ct=storytelling&&from=11.11.2023&to=11.11.2023
Articles count:  20
Found a next page. R

In [179]:
articles_parsed = 0
articles_inserted = 0

for article_href in articles:
    time.sleep(random.uniform(1, 3))
    print("Scraping: ", article_href)
    article_data = scraper_faz.parse_article(article_href)
    print("Parsed: ", article_data)
    articles_parsed += 1
    try:
        mongo_articles.insert_one(article_data)
        print("Inserted instance into database.")
        articles_inserted += 1
    except Exception as ex:
        print("Unable to persist instance: ", ex)

Scraping:  https://www.faz.net/suche/2-bundesliga-hertha-verpasst-mit-2-2-gegen-ksc-anschluss-nach-oben-19263289.html
Parsed:  {'_id': '-gtn-bgvnd', 'keywords': ['Bundesliga', 'Karlsruher SC', 'Hertha BSC Berlin', 'Hertha BSC', 'Pal Dardai', 'Fabian Reese', 'Leon Jensen'], 'premiumArticle': False, 'hrefPermalink': 'https://www.faz.net/-gtn-bgvnd', 'lastUpdate': datetime.datetime(2023, 11, 11, 23, 9, 36, tzinfo=tzoffset(None, 3600)), 'title': '2. Bundesliga: Hertha verpasst mit 2:2 gegen KSC Anschluss nach oben', 'introText': 'Hertha BSC dreht das Spiel gegen den Karlsruher SC zunächst, schafft es am Ende zu Hause aber nicht über ein 2:2 hinaus. Damit liegen die Berliner sechs Punkte hinter dem Relegationsplatz.', 'paragraphs': ['Hertha BSC bleibt in der zweiten Fußball-Bundesliga weiter im Tabellenmittelfeld stecken. Das Team von Trainer Pal Dardai kam in seinem stimmungsvollem Heimspiel gegen den Karlsruher SC nicht über ein 2:2 (2:1) hinaus und liegt weiter satte sechs Punkte hinter 

In [180]:
execution["articles_parsed"] = articles_parsed
execution["articles_inserted"] = articles_inserted

mongo_execs.insert_one(execution)